In [8]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

# Take the sign names in our collections data

In [9]:
signs = sorted(os.listdir("./Keypoint_Data/")) # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

In [10]:
signs[0]

'hello'

In [11]:
sign_id['hello']

0

# Create the our x and y data


In [12]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [16]:
x_train.shape

(950, 30, 1662)

In [17]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(64, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [18]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [19]:
model.fit(x_train, y_train, epochs=1000) # run this line when data collection is done (this will train the model)
# we can start the number of epochs to be equal to 175 to avoid over training and add afterwards if needed

Epoch 1/1000
30/30 [==============================] - 5s 48ms/step - loss: 2.2936 - categorical_accuracy: 0.1147
Epoch 2/1000
30/30 [==============================] - 1s 48ms/step - loss: 2.3338 - categorical_accuracy: 0.1274
Epoch 3/1000
30/30 [==============================] - 1s 46ms/step - loss: 2.2902 - categorical_accuracy: 0.1358
Epoch 4/1000
30/30 [==============================] - 1s 49ms/step - loss: 2.2208 - categorical_accuracy: 0.1779
Epoch 5/1000
30/30 [==============================] - 1s 47ms/step - loss: 2.3010 - categorical_accuracy: 0.1074
Epoch 6/1000
30/30 [==============================] - 1s 48ms/step - loss: 2.2813 - categorical_accuracy: 0.1253
Epoch 7/1000
30/30 [==============================] - 1s 48ms/step - loss: 2.0674 - categorical_accuracy: 0.1958
Epoch 8/1000
30/30 [==============================] - 1s 49ms/step - loss: 1.5399 - categorical_accuracy: 0.3411
Epoch 9/1000
30/30 [==============================] - 1s 49ms/step - loss: 0.9752 - categorical_

KeyboardInterrupt: 

In [6]:
del model

NameError: name 'model' is not defined

In [15]:
model.summary() # summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            408320    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 30, 64)            49408     
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 32)                61472     
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 12)                7

In [20]:
res = model.predict(x_test)

2/2 [==============================] - 0s 10ms/step


In [21]:
signs[np.argmax(res[10])]

'what is your name'

In [22]:
signs[np.argmax(y_test[10])]

'what is your name'

# save weights


In [24]:
model.save("VertoMotus.h5") # save model

In [25]:
model = tf.keras.models.load_model("./VertoMotus.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite = converter.convert()

open("./VertoMotus.tflite", "wb").write(tflite)

INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmpagvi40mu\assets


INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmpagvi40mu\assets


652624

In [38]:
interpreter = tf.lite.Interpreter("LiteML.tflite")

inp = interpreter.get_input_details()
out = interpreter.get_output_details()

interpreter.set_tensor(inp[0]["index"], np.expand_dims(sequences,axis=0))
interpreter.invoke()
                
                #for i in sequence[-30:]:
output_data = interpreter.get_tensor(out[0]["index"])
                
highest = np.argmax(output_data)
word = VM.get_key(signs, highest)
print(output_data.shape)
print(word)


ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_lstm_9_input:0 